In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from sklearn.metrics import mean_squared_error

In [2]:
# Load Metadata
metadata_path = './metadata_aug.csv'
metadata = pd.read_csv(metadata_path)

# Load Images and Labels
image_paths = metadata['path'].values
all_labels = metadata[['coord_x', 'coord_y']].values

# Load images and convert them to arrays
images = []
labels = []
for i, path in enumerate(image_paths):
    if(path[7:12] == 'plume'): #Only train on images that have plume
        path = "./" + path + ".tif"
        img = load_img(path, color_mode='grayscale', target_size=(64, 64))
        img_array = img_to_array(img)
        img_array /= np.max(img_array) #Renormalize to 0,1 because the .tif format has high pixel values 
        images.append(img_array)
        labels.append(all_labels[i])
    

images = np.array(images)
labels = np.array(labels)

# Preprocess Data
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [3]:
# Model Architecture
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2))  # 2 output neurons for x and y coordinates

model.compile(optimizer='adam', loss='mse')  # Mean Squared Error as the loss function for regression


In [4]:
# Train the Model
model.fit(X_train, y_train, epochs=15, batch_size=16, validation_data=(X_val, y_val))

Epoch 1/15
38/38 [==============================] - 5s 81ms/step - loss: 624.0635 - val_loss: 381.2783
Epoch 2/15
38/38 [==============================] - 3s 68ms/step - loss: 380.5828 - val_loss: 305.3297
Epoch 3/15
38/38 [==============================] - 3s 67ms/step - loss: 344.0085 - val_loss: 292.3805
Epoch 4/15
38/38 [==============================] - 2s 64ms/step - loss: 323.8742 - val_loss: 312.9113
Epoch 5/15
38/38 [==============================] - 2s 65ms/step - loss: 312.4187 - val_loss: 264.4364
Epoch 6/15
38/38 [==============================] - 3s 71ms/step - loss: 289.4874 - val_loss: 263.9958
Epoch 7/15
38/38 [==============================] - 3s 69ms/step - loss: 271.3947 - val_loss: 253.0863
Epoch 8/15
38/38 [==============================] - 3s 70ms/step - loss: 260.0743 - val_loss: 263.4184
Epoch 9/15
38/38 [==============================] - 2s 65ms/step - loss: 248.4928 - val_loss: 246.4610
Epoch 10/15
38/38 [==============================] - 3s 67ms/step - loss:

In [7]:
# Make predictions
predictions = model.predict(X_test)

mse = mean_squared_error(y_test, predictions)

# Distance threshold in pixels to seperate correct and wrong predictions
threshold = 20

# Calculate accuracy
accuracy = np.mean(np.all(np.square(y_test - predictions) < threshold**2, axis=1))

print(f'Mean Squared Error: {mse}')
print(f'Test Accuracy: {accuracy}')


5/5 [==============================] - 0s 28ms/step
Mean Squared Error: 228.26510343697177
Test Accuracy: 0.7076923076923077


In [6]:
model.save('point_predictor.h5')

C:\Users\Hp\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
